In [1]:
# Load Libraries

import os
import sys
import copy
import joblib
import pandas as pd
import numpy as np
import mlflow
import time
import warnings
from sklearn.preprocessing import Normalizer, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from IPython.display import Image

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import recall_score, f1_score, accuracy_score, precision_score
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix as con_mat
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


import causalnex
from causalnex.structure.notears import from_pandas
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
from causalnex.discretiser import Discretiser
from causalnex.structure import DAGRegressor
from causalnex.inference import InferenceEngine
from causalnex.network import BayesianNetwork
from causalnex.network.sklearn import BayesianNetworkClassifier
from causalnex.discretiser.discretiser_strategy import (DecisionTreeSupervisedDiscretiserMethod,)
from causalnex.network import BayesianNetwork
from causalnex.inference import InferenceEngine

warnings.filterwarnings("ignore")

/home/nhgirma/anaconda3/envs/10acadenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
merged_df = pd.read_csv('../data/merged.csv')

In [3]:
sys.path.append('../scripts')
from helpers import Helper
UTIL = Helper()

In [4]:
merged_df["date"]=merged_df["trip_start_time"].apply(lambda x: pd.to_datetime(x).date())
merged_df["hour"]=merged_df["trip_start_time"].apply(lambda x: pd.to_datetime(x).hour)

In [5]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 21 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1557740 non-null  int64  
 1   order_id          1557740 non-null  int64  
 2   driver_id         1557740 non-null  int64  
 3   driver_action     1557740 non-null  object 
 4   lat               1557740 non-null  float64
 5   lng               1557740 non-null  float64
 6   trip_id           1557740 non-null  int64  
 7   trip_start_time   1557740 non-null  object 
 8   trip_end_time     1557740 non-null  object 
 9   trip_origin_lat   1557740 non-null  float64
 10  trip_origin_long  1557740 non-null  float64
 11  trip_destn_lat    1557740 non-null  float64
 12  trip_destn_long   1557740 non-null  float64
 13  trip_distance     1557740 non-null  float64
 14  trip_duration     1557740 non-null  float64
 15  speed(kmph)       1557740 non-null  float64
 16  

In [6]:

# merged_df['is_holiday'] = merged_df['trip_start_time'].apply(lambda x: UTIL.is_holiday(x))

In [7]:
# merged_df['is_holiday'].value_counts()

In [9]:
merged_df['is_weekend'] = merged_df['trip_start_time'].apply(lambda x: UTIL.isWeekend(x))

In [12]:
merged_df['is_weekend'].unique()

array([0, 1])

In [17]:
# merged_df[merged_df['trip_duration']<=10]
merged_df["fulfilled"] = (merged_df["driver_action"] == "accepted") & (merged_df["trip_duration"] >=10)

In [19]:
merged_df.head(3)

,id,order_id,driver_id,driver_action,lat,lng,trip_id,trip_start_time,trip_end_time,trip_origin_lat,...,trip_distance,trip_duration,speed(kmph),year,month,day_of_week,date,hour,is_weekend,fulfilled
0,1,392001,243828,accepted,6.602207,3.270465,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,6.601042,...,20.984319,3.0,419.686381,2021,7,Thursday,2021-07-01,9,0,False
1,2,392001,243588,rejected,6.592097,3.287445,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,6.601042,...,20.984319,3.0,419.686381,2021,7,Thursday,2021-07-01,9,0,False
2,3,392001,243830,rejected,6.596133,3.281784,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,6.601042,...,20.984319,3.0,419.686381,2021,7,Thursday,2021-07-01,9,0,False


In [21]:
!pip install geopy
from geopy.distance import great_circle as GRC
# add proximity feature
merged_df['driver_proximity'] = merged_df.apply(lambda x: GRC( (x['lat'], x['lng']),(x['trip_origin_lat'], x['trip_origin_long'])).m, axis=1)


  Using cached geopy-2.2.0-py3-none-any.whl (118 kB)
  Using cached geographiclib-1.52-py3-none-any.whl (38 kB)


In [22]:
merged_df['driver_proximity']

0           693.616345
1          1554.088493
2           788.354435
3           694.071952
4          1623.331351
              ...     
1557735    3276.008974
1557736    3403.414221
1557737    2999.369716
1557738    3276.008974
1557739    3624.744473
Name: driver_proximity, Length: 1557740, dtype: float64